In [ ]:
!pip install 'optimum[onnxruntime-gpu]'
!pip install transformers

In [ ]:
# !pip install tf2onnx
# import tf2onnx

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [21]:
# %pwd
%cd "/content/gdrive/MyDrive/Transformers_files"

/content/gdrive/MyDrive/Transformers_files


## Cargar el transformers y convertirlo a objeto onnx. Guardar

In [25]:
from pathlib import Path
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForQuestionAnswering

# model_id = "deepset/roberta-base-squad2"
model_id = 'scratch_roberta_squad'
onnx_path = Path("onnx_scratch_roberta")

task = "question-answering"


In [26]:
# load vanilla transformers and convert to onnx
model = ORTModelForQuestionAnswering.from_pretrained(model_id, from_transformers = True)
# tokenizer = AutoTokenizer.from_pretrained(model_id)

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

All the layers of TFRobertaForQuestionAnswering were initialized from the model checkpoint at scratch_roberta_squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForQuestionAnswering for predictions without further training.


In [28]:
# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

('onnx_scratch_roberta/tokenizer_config.json',
 'onnx_scratch_roberta/special_tokens_map.json',
 'onnx_scratch_roberta/vocab.json',
 'onnx_scratch_roberta/merges.txt',
 'onnx_scratch_roberta/added_tokens.json',
 'onnx_scratch_roberta/tokenizer.json')

## Optimizar el modelo

In [29]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# create ORTOptimizer and define optimization configuration
optimizer = ORTOptimizer.from_pretrained(model)
optimization_config = OptimizationConfig(optimization_level=99) # enable all optimizations

# apply the optimization configuration to the model
optimizer.optimize(
    save_dir=onnx_path,
    optimization_config=optimization_config) 

PosixPath('onnx_scratch_roberta')

## Después de optimizar el modelo, podemos acelerarlo más con  cuantizándolo con ORTQuantizer

In [30]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# create ORTQuantizer and define quantization configuration
dynamic_quantizer = ORTQuantizer.from_pretrained(model)
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

# apply the quantization configuration to the model
model_quantized_path = dynamic_quantizer.quantize(
    save_dir=onnx_path,
    quantization_config=dqconfig)

### Revisar diferencia de pesos de los modelos

In [31]:
import os

# get model file size
size = os.path.getsize(onnx_path / "model_optimized.onnx")/(1024*1024)
quantized_model = os.path.getsize(onnx_path / "model_quantized.onnx")/(1024*1024)

print(f"Model file size: {size:.2f} MB")
print(f"Quantized Model file size: {quantized_model:.2f} MB")

Model file size: 473.67 MB
Quantized Model file size: 231.00 MB


## Importar el modelo


In [35]:
model = ORTModelForQuestionAnswering.from_pretrained(onnx_path,file_name="model_quantized.onnx")
tokenizer = AutoTokenizer.from_pretrained(onnx_path)
# from transformers import AutoModelForQuestionAnswering
# model = AutoModelForQuestionAnswering.from_pretrained(model_id, from_tf = True)
# tokenizer = AutoTokenizer.from_pretrained(model_id)

In [37]:
modelo = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [ ]:
modelo('pregunta acá','contexto acá')